#### A 3D radial (sphere) simulation

In [1]:
include("../resk.jl")
include("../reskplots.jl")

RESK successfully loaded.
RESKPlots successfully loaded.


In [ ]:
A = deserialize("data/3d_2.re")
A_fitn_frontav = average_front(A,"fitn";oneside=false)
A_fitn_frontav_mean = average_ts(A_fitn_frontav, A["stats"]["n_gens"])
A_fitn_frontav_meanN = norm_onset_mean(A_fitn_frontav_mean, A["stats"]["n_gens_burnin"]+1)
Anb = deserialize("data/3d_2nb.re")
Anb_fitn_frontav = average_front(Anb,"fitn";oneside=false)
Anb_fitn_frontav_mean = average_ts(Anb_fitn_frontav, Anb["stats"]["n_gens"])

In [5]:
function average_front(data, n_gens, x_max, y_max, z_max; greaterzero=false, oneside=false, divide=true)
    n_re = size(data, 5)
    av_arr = Array{Float32}(undef, n_gens, n_re)

    for i in 1:n_re, j in 1:n_gens
        a_sum = 0
        cnt = 0
        # scanning every xy: side 1
        for _x in 1:x_max, _y in 1:y_max
            frontier_z = z_max
            while frontier_z != 1 && (isnan(data[_x, _y, frontier_z, j, i]) || (greaterzero && data[_x, _y, frontier_z, j, i] == 0))
                frontier_z -= 1
            end
            if data[_x, _y, frontier_z, j, i] >= 0 || (greaterzero && data[_x, _y, frontier_z, j, i] > 0)
                a_sum += data[_x, _y, frontier_z, j, i]
                cnt += 1
            end
        end
        # scanning every xy: side 2
        if !oneside
            for _x in 1:x_max, _y in 1:y_max
                frontier_z = 1
                while frontier_z != z_max && (isnan(data[_x, _y, frontier_z, j, i]) || (greaterzero && data[_x, _y, frontier_z, j, i] == 0))
                    frontier_z += 1
                end
                if data[_x, _y, frontier_z, j, i] >= 0 || (greaterzero && data[_x, _y, frontier_z, j, i] > 0)
                    a_sum += data[_x, _y, frontier_z, j, i]
                    cnt += 1
                end
            end
        end
        mean_both_sides_xy = a_sum
        if divide
            mean_both_sides_xy /= cnt
        end

        if !oneside
            a_sum = 0
            cnt = 0
            # scanning every yz: side 1
            for _y in 1:y_max, _z in 1:z_max
                frontier_x = x_max
                while frontier_x != 1 && (isnan(data[frontier_x, _y, _z, j, i]) || (greaterzero && data[frontier_x, _y, _z, j, i]== 0))
                    frontier_x -= 1
                end
                if data[frontier_x, _y, _z, j, i] >= 0 || (greaterzero && data[frontier_x, _y, _z, j, i] > 0)
                    a_sum += data[frontier_x, _y, _z, j, i]
                    cnt += 1
                end
            end
            # scanning every yz: side 2
            for _y in 1:y_max, _z in 1:z_max
                frontier_x = 1
                while frontier_x != x_max && (isnan(data[frontier_x, _y, _z, j, i]) || (greaterzero && data[frontier_x, _y, _z, j, i] == 0))
                    frontier_x += 1
                end
                if data[frontier_x, _y, _z, j, i] >= 0 || (greaterzero && data[frontier_x, _y, _z, j, i] > 0)
                    a_sum += data[frontier_x, _y, _z, j, i]
                    cnt += 1
                end
            end
            mean_both_sides_yz = a_sum
            if divide
                mean_both_sides_yz /= cnt
            end

            a_sum = 0
            cnt = 0
            # scanning every xz: side 1
            for _x in 1:x_max, _z in 1:z_max
                frontier_y = y_max
                while frontier_y != 1 && (isnan(data[_x, frontier_y, _z, j, i]) || (greaterzero && data[_x, frontier_y, _z, j, i] == 0))
                    frontier_y -= 1
                end
                if data[_x, frontier_y, _z, j, i] >= 0 || (greaterzero && data[_x, frontier_y, _z, j, i] > 0)
                    a_sum += data[_x, frontier_y, _z, j, i]
                    cnt += 1
                end
            end
            # scanning every yz: side 2
            for _x in 1:x_max, _z in 1:z_max
                frontier_y = 1
                while frontier_y != y_max && (isnan(data[_x, frontier_y, _z, j, i]) || (greaterzero && data[_x, frontier_y, _z, j, i] == 0))
                    frontier_y += 1
                end
                if data[_x, frontier_y, _z, j, i] >= 0 || (greaterzero && data[_x, frontier_y, _z, j, i] > 0)
                    a_sum += data[_x, frontier_y, _z, j, i]
                    cnt += 1
                end
            end
            mean_both_sides_xz = a_sum
            if divide
                mean_both_sides_xz /= cnt
            end

            av_arr[j, i] = (mean_both_sides_xy + mean_both_sides_yz + mean_both_sides_xz)/3
        else
            av_arr[j, i] = mean_both_sides_xy
        end
    end
    return av_arr
end

average_front (generic function with 4 methods)

In [19]:
function norm_onset_mean2(re::Dict, dataname::String="fitn", offset::Int=0)
    norm_onset_mean2(re[dataname], re["stats"]["n_gens_burnin"], offset)
end

function norm_onset_mean2(data, n_gens_burnin::Int, offset::Int=0)
    normal_array = copy(data)
    n_re = size(data)[end]
    wlddim = length(size(data)) - 2

    for rep in 1:n_re
        sum = 0
        count = 0

        for u in data[repeat([:], wlddim)..., n_gens_burnin + 1 + offset, rep]
            if u > 0
                sum += u
                count += 1
            end
        end

        gen_average = sum / count
        println(gen_average)

        for i in (n_gens_burnin + 1 + offset):size(data, wlddim + 1)
            normal_array[repeat([:], wlddim)..., i, rep] ./= gen_average
        end
    end

    return normal_array
end

norm_onset_mean2 (generic function with 5 methods)

In [ ]:
Plots.plot(A_fitn_frontav_meanN[(A["stats"]["n_gens_burnin"]+2):end],label="Onset mean normalisation",xlabel="Generation",ylims=(0.4,1.1))
Plots.plot!(Anb_fitn_frontav_mean,label="No burn-in")